# Generating Vault Diagrams

`graphviz` is required to output digrams.

You may need to `brew install graphviz` in a console prior to using this notebook.

This Notebook uses [diagrams](https://diagrams.mingrammer.com/docs/getting-started/installation), to define and create diagrams in python code.


## Troubleshooting

Possible tips

### Interpreter Errors
If you encounter a system interpreter or `zsh` error such as 

```shell
zsh: /usr/local/bin/jupyter: bad interpreter: /usr/local/opt/python/bin/python3.7: no such file or directory
```

The brew symlinks may have been overwritten during an update to python.

These instructions may help:

```shell
rm '/usr/local/bin/jupyter'
brew install jupyter
brew link --overwrite jupyter
brew link --overwrite --dry-run jupyter
brew unlink jupyter && brew link jupyter
```



In [ ]:
import sys
!{sys.executable} -m pip install diagrams

In [ ]:
from diagrams import Cluster, Diagram, Edge
from diagrams.aws.compute import ECS, EC2
from diagrams.aws.network import Route53, ELB, ClientVpn, NATGateway, PrivateSubnet, PublicSubnet, VPC

In [ ]:
with Diagram("VPN Networking Overview", show=False) as vpn_diag:
    vault_dns = Route53("DNS\nhttps://production-vault.styleseat.net")
    tech_onboarding_dns = Route53("DNS\nhttps://production-tech-onboarding.styleseat.net")   
  
    with Cluster("VPC"):
        
        client_vpn = ClientVpn("SSO VPN")
        
        with Cluster("Private Subnets\nus-east-1b-ss-app\nus-east-1c-ss-app"):
            with Cluster("Security Groups\nsg-bb212fc9 - us-east-1.ecs.prod\nus-east-1-production-client-vpn-access"):
                with Cluster("AutoScalingGroup"):
                    ec2 = [
                        EC2("production-instance")   
                    ]
        
        with Cluster("Vault\nECS Tasks"):
            vault_ecs = [
                ECS("vault"),
                ECS("vault-init")
            ]
            
        with Cluster("Tech Onboarding\nECS Tasks"):
            tech_onboarding_ecs = ECS("tech-onboarding")
            
        with Cluster("Security Group\nweb-https-lb"):
            public_alb = ELB("production-external-alb")
            private_alb = ELB("production-internal-alb")
            


    # Vault DNS/Networking Flow
    vault_dns >> Edge(xlabel="Port\n443") >> \
        client_vpn >> Edge(xlabel="Port\n443") >> \
        private_alb >> Edge(xlabel="Port\n8200") >> \
        ec2 >> Edge(xlabel="Port\n8200") >> \
        vault_ecs[0]
    
    # Tech Onboarding DNS/Networking Flow
    tech_onboarding_dns >> Edge(xlabel="Port\n443") >> \
        public_alb >> Edge(xlabel="Port\n5556") >> \
        ec2 >> Edge(xlabel="Port\n5556") >> \
        tech_onboarding_ecs
    
    vault_ecs[0] - Edge(style="dashed") - vault_ecs[1]
    
vpn_diag